# 0) Dataset 

We are going to be working with this dataset: [Machine Predictive Maintenance Classification](https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification).

# 1) Read Dataset

In [21]:
import pandas as pd
from funpymodeling.exploratory import status
import mlflow.sklearn

In [22]:
pd.set_option('display.max_columns', None)

In [23]:
mlflow.set_experiment(experiment_name="proyecto_bootcamp_jueves")

2024/10/04 17:46:36 INFO mlflow.tracking.fluent: Experiment with name 'proyecto_bootcamp_jueves' does not exist. Creating a new experiment.


<Experiment: artifact_location=('/home/alumni/Programación/Bootcamp Data Science y '
 'MLops/Material/Semana9/Tarea/src/notebook/../../deployment/mlruns/899917425650450696'), creation_time=1728074796805, experiment_id='899917425650450696', last_update_time=1728074796805, lifecycle_stage='active', name='proyecto_bootcamp_jueves', tags={}>

In [24]:
mlflow.set_tracking_uri('../../deployment/mlruns/')

In [25]:
data = pd.read_csv("../datasets/raw_data/predictive_maintenance.csv", sep=',') 

In [26]:
data.head(5)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


# 2) Data Preparation

In [27]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,UDI,0,0.0,0,0.0000,10000,int64
1,Product ID,0,0.0,0,0.0000,10000,object
2,Type,0,0.0,0,0.0000,3,object
3,Air temperature [K],0,0.0,0,0.0000,93,float64
4,Process temperature [K],0,0.0,0,0.0000,82,float64
5,Rotational speed [rpm],0,0.0,0,0.0000,941,int64
6,Torque [Nm],0,0.0,0,0.0000,577,float64
7,Tool wear [min],0,0.0,120,0.0120,246,int64
8,Target,0,0.0,9661,0.9661,2,int64
9,Failure Type,0,0.0,0,0.0000,6,object


In [28]:
data = data.drop(['UDI', 'Product ID', 'Failure Type'], axis=1)

In [29]:
data.rename(
    columns = {
        'Air temperature [K]':'Air_temperature',
        'Process temperature [K]':'Process_temperature',
        'Rotational speed [rpm]':'Rotational_speed',
        'Torque [Nm]':'Torque',
        'Tool wear [min]':'Tool_wear',
        }, 
    inplace = True)

In [30]:
data.head(5)

,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Target
0,M,298.1,308.6,1551,42.8,0,0
1,L,298.2,308.7,1408,46.3,3,0
2,L,298.1,308.5,1498,49.4,5,0
3,L,298.2,308.6,1433,39.5,7,0
4,L,298.2,308.7,1408,40.0,9,0


In [31]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Type,0,0.0,0,0.0000,3,object
1,Air_temperature,0,0.0,0,0.0000,93,float64
2,Process_temperature,0,0.0,0,0.0000,82,float64
3,Rotational_speed,0,0.0,0,0.0000,941,int64
4,Torque,0,0.0,0,0.0000,577,float64
5,Tool_wear,0,0.0,120,0.0120,246,int64
6,Target,0,0.0,9661,0.9661,2,int64


In [32]:
data = pd.get_dummies(data)

In [33]:
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Air_temperature,0,0.0,0,0.0000,93,float64
1,Process_temperature,0,0.0,0,0.0000,82,float64
2,Rotational_speed,0,0.0,0,0.0000,941,int64
3,Torque,0,0.0,0,0.0000,577,float64
4,Tool_wear,0,0.0,120,0.0120,246,int64
5,Target,0,0.0,9661,0.9661,2,int64
6,Type_H,0,0.0,8997,0.8997,2,bool
7,Type_L,0,0.0,4000,0.4000,2,bool
8,Type_M,0,0.0,7003,0.7003,2,bool


In [34]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Air_temperature,10000.0,300.00493,2.000259,295.3,298.3,300.1,301.5,304.5
Process_temperature,10000.0,310.00556,1.483734,305.7,308.8,310.1,311.1,313.8
Rotational_speed,10000.0,1538.77610,179.284096,1168.0,1423.0,1503.0,1612.0,2886.0
Torque,10000.0,39.98691,9.968934,3.8,33.2,40.1,46.8,76.6
Tool_wear,10000.0,107.95100,63.654147,0.0,53.0,108.0,162.0,253.0
Target,10000.0,0.03390,0.180981,0.0,0.0,0.0,0.0,1.0


### Save processed dataset

In [35]:
data.to_csv("../datasets/processed_data/processed_data.csv",sep=",")

# 3) Classification Model

## 3.1 - Separate between train and test 

In [36]:
data_x = data.drop('Target', axis=1)
data_y = data['Target']

In [38]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.3

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=TEST_SIZE )

## 3.2 - Train model

In [39]:
from sklearn.ensemble import RandomForestClassifier 

N_ESTIMATORS = 5000
RANDOM_STATE = 19

rf_model = RandomForestClassifier(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE)


In [40]:
rf_model.fit(x_train,y_train)

RandomForestClassifier(n_estimators=5000, random_state=19)

## 3.3 - Metrics

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
pred_ts = rf_model.predict(x_test)
pred_tr = rf_model.predict(x_train)
pred_ts

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
pred_tr

array([0, 0, 1, ..., 0, 1, 0])

In [44]:
df_val_tr=pd.DataFrame({'y_train':y_train, 'pred_tr':pred_tr})
df_val_ts=pd.DataFrame({'y_test':y_test, 'pred_ts':pred_ts})

In [45]:
df_val_tr

,y_train,pred_tr
3104,0,0
2437,0,0
4044,1,1
9182,0,0
7178,0,0
...,...,...
8085,0,0
9957,0,0
5167,0,0
4761,1,1


In [46]:

accuracy_test = accuracy_score(df_val_ts.y_test,df_val_ts.pred_ts, normalize=True)
accuracy_train = accuracy_score(df_val_tr.y_train,df_val_tr.pred_tr, normalize=True)

In [47]:
accuracy_test

0.9866666666666667

# 4. Save model  and ohe_columns

In [48]:
import pickle

In [49]:
pickle.dump(data_x.columns,open("../ohe_categories.pkl","wb"),protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
pickle.dump(rf_model,open("../model/rf.pkl","wb"))

# 5. Save parameters, model and metrics with MLflow

## 5.1 - Save metrics

In [51]:
mlflow.log_metric("Train Accuracy ", accuracy_train)

mlflow.log_metric("Test Accuracy ", accuracy_test)

## 5.2 - Save parameters

In [52]:
mlflow.log_param("Dataset size", data.shape)

mlflow.log_param("Percentage of test size", TEST_SIZE)

mlflow.log_param("Number of estimators", N_ESTIMATORS)

mlflow.log_param("Random state", RANDOM_STATE)

19

## 5.3 - Save model

In [53]:
mlflow.sklearn.log_model(rf_model,"My_model")

2024/10/04 17:47:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
